In [1]:
from scipy.spatial import Delaunay
from itertools import combinations
import numpy as np
def delune():
    # read data
    data = [] 
    cnt = 0
    f = open('3oxc.filtered.pdb')
    for line in f:
        cnt += 1
        data.append(line.split())
    assert cnt == 198
    f.close()
    
    # get the positions and corresponding coordinates
    # place = [int(d[5]) for d in data] # 1.. 99, 101.. 199] 
    x_y_z = [[float(x) for x in d[6:9]] for d in data]
    x_y_z_place = {tuple(x_y_z[i]):i for i in range(len(x_y_z))}
    
    # triangulate 
    tri = Delaunay(x_y_z)
    all_combs = list(combinations(range(4),2))
    
    # storage 
    place_neighbor_dist = {i:{} for i in range(0,198)}
    
    # get the distances 
    for n in np.array(x_y_z)[tri.simplices]:
        for c in all_combs:
            first, second = n[c[0]], n[c[1]]
            dist = np.linalg.norm(first - second) # l2 norm
            p1, p2 = x_y_z_place[tuple(first)], x_y_z_place[tuple(second)]

            place_neighbor_dist[p1].update({p2:dist})
            place_neighbor_dist[p2].update({p1:dist})
    
    return place_neighbor_dist

In [2]:
place_neighbor_dist = delune()

In [3]:
amino_acids = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
aa_ind = {p:i for i,p in enumerate(amino_acids)}
vec_ind = {v:k for k,v in enumerate([(amino_acids[i], amino_acids[j]) for i in range(len(amino_acids)) for j in range(i,len(amino_acids))])}

In [4]:
def get_vector(line):
    vec_sum = np.zeros(210)
    vec_tot = np.zeros(210)
    for i,aa in enumerate(line):
        p1 = aa_ind[aa]
        n_d = place_neighbor_dist[i]
        for n,d in n_d.items():
            naa = line[n]
            p2 = aa_ind[naa]
            pair = (aa, naa) if aa < naa else (naa, aa)
            pair_ind = vec_ind[pair]
            vec_sum[pair_ind] += d
            vec_tot[pair_ind] += 1
    vec_tot = map(lambda x:1 if x == 0 else x,vec_tot)
    return vec_sum/vec_tot

In [6]:
from collections import defaultdict
import time 
filetowrite = 'PI_DataSet_6_19_PI_avg_dist_vec_210.txt'
g = open(filetowrite,'w')
f = open("PI_DataSet_6_19_expanded.txt")
for line in f:
    header = line.split("\t")
    vec_start = header.index("P1")
    break
count = 0
t0 = time.time()
for line in f:
    line = line.strip().split("\t")
    l = line[:vec_start]
    v =   line[vec_start:]
    vec = get_vector(2*v)
    vec = map(lambda x:'{0:.5f}'.format(x),vec)
    token = "\t".join(l+vec)+"\n"
    g.write(token)
    count += 1
    if count % 10000 == 0:
        print count, time.time() - t0
        t0 = time.time()
print(count)

10000 23.0414340496
20000 22.0587670803
30000 22.0212550163
40000 22.8803281784
50000 23.1843428612
60000 23.0212371349
70000 21.9156498909
80000 22.7735869884
90000 21.9916410446
100000 21.7946491241
110000 22.8187689781
120000 22.9277729988
130000 22.3492059708
140000 22.9786520004
150000 22.5455210209
160000 23.3900208473
170000 22.8075900078
180000 22.3402950764
190000 22.9610199928
200000 22.7608480453
210000 22.5171790123
220000 22.1702740192
230000 22.0521349907
240000 22.012804985
250000 22.07279706
260000 22.1068749428
270000 22.1800839901
280000 22.0646219254
290000 23.1454520226
300000 23.0171051025
310000 22.0555670261
320000 22.4320130348
330000 22.2875871658
340000 28.0314459801
350000 22.2498009205
360000 22.0087640285
370000 22.0577888489
380000 22.0570859909
390000 22.2293179035
400000 22.0563168526
410000 22.3024859428
414009
